<a href="https://colab.research.google.com/github/dakilaledesma/oai_rag/blob/main/example_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Retrieval Based Augmentation
#### Intro
This is a simple retrieval based augmentation workflow. What it does is:
1. Read an Excel sheet, where each row in a column is a "piece of information"
2. Use algorithms* to determine which rows of information are relevant to the prompt being asked to the model
3. The most relevant rows of information are fed to ChatGPT as context
4. ChatGPT is asked to answer questions based on the context of those most relevant rows of information.

The nice thing is that ChatGPT itself can string pieces of information together to make a coherent answer. As a simple example, if some rows pertain to asthma, and other rows are pertinent to upper respiratory infections, and one asks "How and which upper respiratory infections does asthma exacerbate?" it will draw information from relevant rows together--even if these rows have no overlap of information--to create a single cohesive answer.

*algos are creating embeddings from text using Ada, then using cosine similarity between the prompt asked and the info given

#### Okay I don't care what do I need to do (Instructions)
1. Retrieve an endpoint from OpenAI to be able to fill out details in cell A
2. Upload your Excel sheet (on the left side of this Colab page)
3. Change info in cell B
  - Change the filename to the filename of your file
  - Change info_column to the column in the spreadsheet that contains the information you want to feed to ChatGPT
4. Change info in cell C, instructions given within the cell
5. Go to Runtime -> Run all above (or CTRL+F9/CMD+F9)
6. Go to the bottom cell, change the question to what you want. Any time you want to ask a new question or get a new response re-run only that cell (through the play button next to the cell, or SHIFT+ENTER)


In [ ]:
%%sh
pip uninstall typing_extensions
pip install openai==0.28
pip install tiktoken

In [ ]:
import openai
import tiktoken
import pandas as pd

In [ ]:
# Cell A: OpenAI configuration
openai.api_type = "azure"  # For BCBS this was "azure" as we have an azure endpoint
openai.api_key = "d0781e8d..."  # Will look like a garbled mess, ours was "d0781e8d..." (not sharing the entire thing for obvious reasons)
openai.api_base = "https://somethingsomethingsomething.openai.azure.com/"  # Will be a URL, ours was similar to https://somethingsomethingsomething.openai.azure.com/
openai.api_version = "2023-05-15" # Keep this the same

In [ ]:
# Cell B: Excel Sheet filename and column
# Change this filename to the file name of your excel sheet
df = pd.read_excel("Responses.xlsx")\

# Change this to the column in the Excel sheet where you'd like to feed the GPT
# information
info_column = "Question"

In [ ]:
# Cell C: Prompt "Engineering"

# Change this to how you want the GPT model to answer with your sources of
# information. You will want to be as assertive and direct as you feel is
# necessary. This will depend on how closely you would like the model to adhere
# to these rules. An example may be found below (real example given).
assertion_text = """You are given information on how our company responds to request for proposal questions. Comments, keywords, and short descriptions describe what the question may be about, and responses are response to that question.
Also note that we as a company are trying to differentiate ourselves by highlighting a personal approach that includes consultation, easy reporting solutions, convenience etc. while you formulate potential responses.

Please generate us responses given the above as context, noting that we require the content IDs as sources of how you answer that question (multiple content IDs are fine). Please make sure to answer with Content IDs.
"""

In [ ]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df['n_tokens'] = df[info_column].apply(lambda x: len(tokenizer.encode(x)))

ada_embeddings = []
for _, row in df.iterrows():
    if row["n_tokens"] < 7800:
        response = openai.Embedding.create(
            input=row[info_column],
            engine="text_embedding_ada_002_API_testing"
        )
        ada_embeddings.append(response['data'][0]['embedding'])
    else:
        print(row["Content ID"])
        ada_embeddings.append([0] * 1536)

df["Embeddings"] = ada_embeddings

In [ ]:
_toks = [v for v in rfp_df["n_tokens"] if v < 7800]
_avg = sum(_toks) / len(_toks)

def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        engine="text_embedding_ada_002_API_testing")
    df["similarities"] = df["Embeddings"].apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res

# Question answer part
Ask your question here when you're done setting up the above Excel sheet, settings, etc.

In [ ]:
# Change this to your question
user_prompt = "Give me an appropriate response to this question: Blah blah something about anaesthesia???"

res = search_docs(df, user_prompt, top_n=28)
toks = 0
texts = []
incs = []
for _, row in res.iterrows():
    if toks + row["n_tokens"] > 7800:
        continue
    else:
        toks += row["n_tokens"]

    texts.append(row["Embedding Text"])
    incs.append(row["Content ID"])

# You can change the first part of the following (Give me an appropriate...) if
# there is a more appropriate prompt engineering prompt for your use case
texts.append(f"Give me an appropriate response to this question: {user_prompt}")

full_response = ""
for response in openai.ChatCompletion.create(
    engine="35_turbo",
    model="gpt-3.5-turbo",
    temperature=2.0,
    messages=messages,
    stream=True
):
    resp = response.choices[0].delta.get("content", "")
    full_response += resp

print(full_response)